# named tuple

reference

    https://docs.python.org/ko/3.7/library/collections.html#collections.namedtuple

usage

    collections.namedtuple(typename, field_names, *, rename=False, defaults=None, module=None)¶

>basic

In [1]:
# declare named tuple
from collections import namedtuple
pt = namedtuple(typename='point', field_names=('x', 'y'))


# dict ---> named tuple
origin_dict = {'x' : 0, 'y' : 0}
origin_tuple = pt(**origin_dict)

# create named tuple
origin_tuple2 = pt(0, 0)

# print
print(origin_tuple)
print(origin_tuple2)
print(f'x : {origin_tuple.x} | y : {origin_tuple.y}')

point(x=0, y=0)
point(x=0, y=0)
x : 0 | y : 0


> _make

    create new instance

In [2]:
pt_1_2 = pt._make((1, 2))
print(pt_1_2)
print(f'x : {pt_1_2.x} | y : {pt_1_2.y}')

point(x=1, y=2)
x : 1 | y : 2


> _fields

    look up name of fields

In [3]:
print(pt._fields)

('x', 'y')


> _asdict

    get dictionary from named tuple

In [4]:
print(type(origin_tuple._asdict()))
print(origin_tuple._asdict())

<class 'dict'>
{'x': 0, 'y': 0}


> default value

In [5]:
field_names = ('name', 'age')
Person = namedtuple('Person', field_names=field_names, \
    defaults= ('karma', 28))    # <--- defalut

karma = Person()
veiga = Person('veiga', 30)

print(f"karma ---> name : {karma.name} | age : {karma.age}")
print(f"veiga ---> name : {veiga.name} | age : {veiga.age}")

karma ---> name : karma | age : 28
veiga ---> name : veiga | age : 30


> Jsonify

In [6]:
import json

# named tuple ---> dict
karma_json = json.dumps(karma._asdict())

# dict ---.> json
print(karma_json)

{"name": "karma", "age": 28}


> @dataclass

In [7]:
from dataclasses import dataclass
from typing import Any
	
@dataclass
class Person:
    name: str = 'karma'
    age: 'int > 0' = 28

print(Person()) 

Person(name='karma', age=28)


# Metaclass

### Basic

> type([class name(str)], [super classes(tuple)], [methods(dict)])

example 1

In [8]:
hi = type('hi', (), {})
print(f"hi : {hi}")


hi : <class '__main__.hi'>


example 2

In [9]:
from random import shuffle 

def shuffling(self):
    shuffle(self.cards)

shuffleCards = lambda self : shuffling
MyCard = type('Cards', (), {'description' : '내 카드s', \
    'cards' : [1,4,5,2,3,5,3,4], \
        'shuffle' : shuffling})

In [10]:
c = MyCard()
print(f"shuffle 전 : {c.cards}")

c.shuffle()
print(f"shuffle 후 : {c.cards}")

shuffle 전 : [1, 4, 5, 2, 3, 5, 3, 4]
shuffle 후 : [3, 2, 4, 5, 1, 3, 5, 4]


> inherit type

    class MetaClassName(type):         
        def __new__(metacls, class name, bases, namespace):
            pass

    class MetaClassName(type):         
        def __new__(metacls, *args, **kwargs):
            pass

In [11]:
class Calculator(type):
    def __new__(metaclas, name, bases, namespace):
        namespace['description'] = "계산기 metaclass"
        namespace['add'] = lambda self, x, y : x+y
        namespace['mul'] = lambda self, x, y : x*y
        namespace['power'] = lambda self, x, y : x**y

        return type.__new__(metaclas, name, bases, namespace)

In [12]:
Calc = Calculator('Calc',(), {})
c = Calc()

print(Calc)
print(c)

<class '__main__.Calc'>


In [13]:
print(f"Description : {c.description}")
print(f"1 + 1 = {c.add(1,1)}")
print(f"2^3 = {c.power(2,3)}")

Description : 계산기 metaclass
1 + 1 = 2
2^3 = 8


### Inheritance

상속통제 - 다중상속을 금지

In [14]:
class DisAllowMultiInherit(type):
    def __new__(metacls, *args, **kwargs):
        if len(args[1]) > 1:
            raise Exception('다중 상속됨')
        return super().__new__(metacls, *args, **kwargs) 

class A1(metaclass=DisAllowMultiInherit):pass

class A2:pass

class B(A1, A2):pass

Exception: 다중 상속됨

상속금지 

In [15]:
class DisAllowInherit(type):
    def __new__(metacls, *args, **kwargs):
        if [c for c in args[1] if isinstance(c, metacls)]:
            raise Exception('상속됨')
        return super().__new__(metacls, *args, **kwargs) 

class A1(metaclass=DisAllowInherit):pass

class A2:pass

class B(A1, A2):pass

Exception: 상속됨

Singleton

    overiding __call__ method : 처음 한번만 객체를 생성하고, 다음부터는 생성된 객체를 반환

In [16]:
class Singleton(type):              # inherit type
    __instances = {}                
    def __call__(cls, *args, **kwargs):
        if cls not in cls.__instances:
            cls.__instances[cls] = super().__call__(*args, **kwargs)
        return cls.__instances[cls]

In [17]:
class Hello(metaclass=Singleton):
    def __init__(self, greet : str = "HI"):
        self.greet = greet

In [18]:
a = Hello("안녕")
b = Hello("Hello")

In [19]:
print(a.greet, b.greet)
print(a == b)
print(id(a) == id(b))

안녕 안녕
True
True


In [20]:
a.greet = "안녕하세요~"
print(a.greet)
print(b.greet)

안녕하세요~
안녕하세요~


Validation

In [82]:
class ValidateScore(type):
    def __new__(metacls, clsname, bases, namespace):
        if namespace['score'] != None:
            if (namespace['score'] <0) | (namespace['score']>100):
                raise ValueError("점수는 0~100점 사이")
        return type.__new__(metacls, clsname, bases, namespace)

class Score(metaclass=ValidateScore):
    score = None

class Math(Score):
    score = None

class English(Score):
    score = -10

ValueError: 점수는 0~100점 사이

### Descriptor

    python 3.6버전부터 동작

In [83]:
class MetaClass(type):
    def __new__(metacls, clsname, bases, namespace):
        self = super().__new__(metacls, clsname, bases, namespace)

        for (key, value) in self.__dict__.items():
            func = getattr(value, '__set_name__', None)
            if func != None: # __set__name__ 이라는 attribute를 가지고 있다면 
                func(self, key)
        return self

In [84]:
class Description:
    def __get__(self, instance, owner):
        return instance.__dict__[self.name]

    def __set__(self, instance, value):
        instance.__dict__[self.name] = value

    def __set_name__(self, owner, name):
        self.name = name